# 文本分类 Text Classification
顾名思义，将不同的文件进行分类，
* 比如“生活”类文章、“娱乐”类文章、“体育”类文章；
* 比如含“猫”这个词比较多，权重比较大的文件归为“猫”类；
* 比如根据文件中违禁词的权重，划分等级

分析过程基于一个统计量TF-IDF（也是一个统计（加权）方法）

TF: Term Frequency 词频
IDF: Inverse Document Frequency 逆向文件频率

**Example**：
* TF: 假设一篇文章(一份文件）有100词，其中 cat 出现了3次，则cat 的词频TF 为 3/100 = 0.03；
* IDF：有多少文件中出现过cat 一词，现在假设有 1000篇文章／文件 出现过cat，而文件总数是10,000,000份，则IDF 为 log(10,000,000/1,000) = 4

最后TF-IDF 权重= 0.03 x 4 = 0.12

**TF-IDF用于评估一个词的重要性，重要性与该词在文件中的词频成正比，同时随它在整个文件集（语料库）中出现的频率成反比(比如is、the 这些词)。**

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from sklearn.datasets import fetch_20newsgroups

category_map = {'misc.forsale': 'Sales', 'rec.motorcycles': 'Motorcycles',
               'rec.sport.baseball': 'Baseball', 'sci.crypt': 'Cryptography',
               'sci.space': 'Space'}
training_data = fetch_20newsgroups(subset='train', categories=category_map.keys(), shuffle=True, random_state=7)
# 采用一部分子集，通过指定categories来选取哪部分作为子集，即该训练集中的数据一定属于指定categories 中。默认采用全集，全categories。


### 1、 将文本 数值向量化-文本数据转换为数值数据

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_termcounts = vectorizer.fit_transform(training_data.data) 
print('\nDimensions of training data:',X_train_termcounts.shape)


Dimensions of training data: (2968, 40605)


### 2、文本转化为数值向量后，进行tf-idf 统计，计算每个词的权重，仍然是数据预处理

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_termcounts)


### 3、有了特征向量之后，Training a classifier

In [5]:
from sklearn.naive_bayes import MultinomialNB # 多类别分类
classifier = MultinomialNB().fit(X_train_tfidf, training_data.target)


### 4、Predict the output categories

In [6]:
input_data = [
    "The curveballs of right handed pitchers tend to curve to the left",
    "Caesar cipher is an ancient form of encryption",
    "This two-wheler is really good on slippery roads"
]

# 同样，先对input做预处理：向量化和tfidf
X_input_termcounts = vectorizer.transform(input_data)  # 前面是.fit_transform,这里是.transform，因为已经fit了
X_input_tfidf = tfidf_transformer.transform(X_input_termcounts)

predicted_categories = classifier.predict(X_input_tfidf)

# 打印分类的结果
for sentence, category in zip(input_data, predicted_categories):
    print("\nInput: ", sentence, '\nPredicted category:',category_map[training_data.target_names[category]])


Input:  The curveballs of right handed pitchers tend to curve to the left 
Predicted category: Baseball

Input:  Caesar cipher is an ancient form of encryption 
Predicted category: Cryptography

Input:  This two-wheler is really good on slippery roads 
Predicted category: Motorcycles


第一句话预测为 Baseball 类

第二句话预测为 Cryptography 凯撒密码-密码学